In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00


In [ ]:
  !pip install verovio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 38.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
model = model.eval().cuda()


# input your test image
image_file = r'/content/20250120_170137.jpg'

# plain texts OCR
res = model.chat(tokenizer, image_file, ocr_type='ocr')

# format texts OCR:
# res = model.chat(tokenizer, image_file, ocr_type='format')

# fine-grained OCR:
# res = model.chat(tokenizer, image_file, ocr_type='ocr', ocr_box='')
# res = model.chat(tokenizer, image_file, ocr_type='format', ocr_box='')
# res = model.chat(tokenizer, image_file, ocr_type='ocr', ocr_color='')
# res = model.chat(tokenizer, image_file, ocr_type='format', ocr_color='')

# multi-crop OCR:
# res = model.chat_crop(tokenizer, image_file, ocr_type='ocr')
# res = model.chat_crop(tokenizer, image_file, ocr_type='format')

# render the formatted OCR results:
# res = model.chat(tokenizer, image_file, ocr_type='format', render=True, save_render_file = './demo.html')

print(res)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Complies with IM DAS t and ard s D BI 01104 logitech G. com NASDAQ: LOGI ogitech Asia Pacific Ltd.  Tel: 852-2624-3600 Fax: 852-2520-2230 S/ N: 2440 AP 7 HJ W N 9 910-005293 Tel: 65-3163-2001 Fax: 65-621-5016 G 304 Logitech Australia Periphery s Pty Ltd Tel: 61-2-9764-8500 Fax: 61-2-9642-0151 027 9785513775 007


In [ ]:
# input your test image
image_file = r'/content/20250120_170137.jpg'

# plain texts OCR
res = model.chat(tokenizer, image_file, ocr_type='ocr')

# format texts OCR:
# res = model.chat(tokenizer, image_file, ocr_type='format')

# fine-grained OCR:
# res = model.chat(tokenizer, image_file, ocr_type='ocr', ocr_box='')
# res = model.chat(tokenizer, image_file, ocr_type='format', ocr_box='')
# res = model.chat(tokenizer, image_file, ocr_type='ocr', ocr_color='')
# res = model.chat(tokenizer, image_file, ocr_type='format', ocr_color='')

# multi-crop OCR:
# res = model.chat_crop(tokenizer, image_file, ocr_type='ocr')
# res = model.chat_crop(tokenizer, image_file, ocr_type='format')

# render the formatted OCR results:
# res = model.chat(tokenizer, image_file, ocr_type='format', render=True, save_render_file = './demo.html')

print(res)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Complies with IM DAS t and ard s D BI 01104 logitech G. com NASDAQ: LOGI ogitech Asia Pacific Ltd.  Tel: 852-2624-3600 Fax: 852-2520-2230 S/ N: 2440 AP 7 HJ W N 9 910-005293 Tel: 65-3163-2001 Fax: 65-621-5016 G 304 Logitech Australia Periphery s Pty Ltd Tel: 61-2-9764-8500 Fax: 61-2-9642-0151 027 9785513775 007


In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

class BarcodeSerialDetector:
    def __init__(self):
        # Initialize YOLOv9 for barcode detection
        self.yolo_model = torch.hub.load('ultralytics/yolov8', 'custom', 'yolov8-barcodes.pt')

        # Initialize GOT-OCR
        self.processor = AutoProcessor.from_pretrained("stepfun-ai/GOT-OCR")
        self.ocr_model = AutoModelForVision2Seq.from_pretrained("stepfun-ai/GOT-OCR")

    def detect_barcode(self, image):
        """
        Detect barcodes in the image using YOLOv9
        Returns the coordinates of detected barcodes
        """
        results = self.yolo_model(image)

        # Extract bounding boxes for barcode class
        boxes = []
        for det in results.xyxy[0]:
            if det[-1] == 0:  # Assuming barcode is class 0
                x1, y1, x2, y2 = map(int, det[:4])
                boxes.append((x1, y1, x2, y2))

        return boxes

    def get_serial_region(self, image, barcode_box):
        """
        Extract the region below the barcode where serial number is likely to be
        """
        x1, y1, x2, y2 = barcode_box

        # Define region below barcode (adjust these parameters as needed)
        padding = 20
        height_multiplier = 1.5

        # Calculate the region below barcode
        serial_y1 = y2
        serial_y2 = min(int(y2 + (y2 - y1) * height_multiplier), image.shape[0])
        serial_x1 = max(0, x1 - padding)
        serial_x2 = min(x2 + padding, image.shape[1])

        return image[serial_y1:serial_y2, serial_x1:serial_x2]

    def read_serial_number(self, image_region):
        """
        Use GOT-OCR to read the serial number from the image region
        """
        # Convert to PIL Image if needed
        if isinstance(image_region, np.ndarray):
            image_region = Image.fromarray(image_region)

        # Process image for OCR
        inputs = self.processor(images=image_region, return_tensors="pt")

        # Generate OCR prediction
        outputs = self.ocr_model.generate(**inputs)

        # Decode prediction
        predicted_text = self.processor.batch_decode(outputs, skip_special_tokens=True)[0]

        return predicted_text

    def process_image(self, image_path):
        """
        Main pipeline to process an image and extract serial numbers
        """
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Could not read image")

        # Detect barcodes
        barcode_boxes = self.detect_barcode(image)

        results = []
        for box in barcode_boxes:
            # Get region below barcode
            serial_region = self.get_serial_region(image, box)

            # Read serial number
            serial_number = self.read_serial_number(serial_region)

            results.append({
                'barcode_location': box,
                'serial_number': serial_number
            })

        return results

# Example usage
if __name__ == "__main__":
    detector = BarcodeSerialDetector()

    # Process single image
    image_path =  r'/content/20250120_170137.jpg'
    results = detector.process_image(image_path)

    for idx, result in enumerate(results):
        print(f"Detection {idx + 1}:")
        print(f"Barcode location: {result['barcode_location']}")
        print(f"Serial number: {result['serial_number']}")
        print("---")

HTTPError: HTTP Error 404: Not Found

In [ ]:
!pip install inference

In [ ]:
!pip install supervision

In [ ]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="jMllDEfAHGqPHhDJcN9I"
)

result = CLIENT.infer('20250120_170137.jpg', model_id="yolo-barcode-detection-ctau6/2")
result

{'inference_id': '648dbf7d-affa-4b0d-aa80-33dd23f57c13',
 'time': 0.1883752559997447,
 'image': {'width': 4592, 'height': 8160},
 'predictions': []}

Train barcode yolo

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="jMllDEfAHGqPHhDJcN9I")
project = rf.workspace("yjj-sxvmn").project("yolo-barcode-detection-ctau6")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to YOLO-Barcode-Detection-2 in yolov8:: 100%|██████████| 470/470 [00:00<00:00, 1050.19it/s]


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.4/910.4 kB 22.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from tqdm import tqdm

# Paths to data
DATA_YAML_PATH = r"/content/YOLO-Barcode-Detection-2/data.yaml"  # Replace with your dataset YAML file path
PRETRAINED_MODEL_PATH = "yolov8n.pt"  # Using YOLOv8 nano as the base model
OUTPUT_DIR = "runs/train"

# Initialize the YOLO model
model = YOLO(PRETRAINED_MODEL_PATH)

# Train the YOLO model
train_results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,                # Number of training epochs
    batch=16,                 # Batch size
    imgsz=640,                # Image size
    project=OUTPUT_DIR,       # Output directory
    name="barcode_detection",  # Experiment name
    optimizer="AdamW",       # Optimizer for better convergence
    save_period=1             # Save model after every epoch
)

# Evaluate the trained model on the validation set using tqdm for progress
print("\nEvaluating model on validation data...")
results = []

for val_image in tqdm(model.val(data=DATA_YAML_PATH), desc="Validation Progress"):
    results.append(val_image)

# Save the model with the best accuracy
best_model_path = f"{OUTPUT_DIR}/barcode_detection/weights/best.pt"
model.export(format="torchscript", dynamic=True, path=best_model_path)
print(f"Best model saved to: {best_model_path}")

# Test the YOLO model
TEST_IMAGES_PATH = "../test/images"  # Path to test images
test_results = model.predict(source=TEST_IMAGES_PATH, save_txt=True, save_conf=True)

print("\nTesting completed.")
print(f"Predictions saved to: {OUTPUT_DIR}/barcode_detection/predict")


Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO-Barcode-Detection-2/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=runs/train, name=barcode_detection2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

100%|██████████| 755k/755k [00:00<00:00, 23.4MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 109MB/s]


AMP: checks passed ✅


...

                   all         37         38          1      0.988      0.995       0.79
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/train/barcode_detection2

Evaluating model on validation data...
Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO-Barcode-Detection-2/valid/labels.cache... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.54it/s]


                   all         37         38          1      0.988      0.995       0.78
Speed: 0.3ms preprocess, 19.1ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/train/barcode_detection22


Validation Progress: 0it [00:00, ?it/s]


TypeError: 'DetMetrics' object is not iterable